In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
from datetime import datetime

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.set(rc={'figure.figsize':(12,9)})

In [ ]:
firms.insert(1, 'trend', range(1,len(firms)+1))

In [ ]:
import statsmodels.api as sm

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(model, 'trend', fig=fig)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(model.resid)

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(model.resid)

In [ ]:
firms['month'] = pd.DatetimeIndex(df['Date']).month

In [ ]:
lm = sm.OLS(y.astype(float),X)
model = lm.fit()
model.summary()

In [ ]:
new_df = pd.DataFrame(model.fittedvalues,  index = model.fittedvalues.index)
new_df.columns = ["Fitted"]
new_df["Real"] = firms["Total"]
plt.plot(firms.trend,new_df.Fitted, c = "r")
plt.plot(firms.trend,new_df.Real, c = "b")
plt.show()

In [ ]:
lag = firms["Total"].shift(-1)
firms["lag"] = lag
firms.drop(firms.tail(1).index, inplace = True)

In [ ]:
index = cons["Date"]+ " " +cons["Hour"]
hourly_date = pd.to_datetime(index, format = "%d/%m/%Y %H:%M")
cons.rename(columns={"Consumption (MWh)": "Consumption"}, inplace = True)
cons.set_index(hourly_date, inplace = True)
cons["Date"] = pd.to_datetime(cons["Date"], format = "%d/%m/%Y")
cons['Day'] = cons['Date'].dt.day_name()
cons['Month'] = cons['Date'].dt.month_name()
cons['Week'] = cons.Date.apply(lambda x : x.weekofyear)

In [ ]:
sns.lineplot(x = cons.index, y = "Consumption", hue = "Hour", data = cons);

In [ ]:
sns.set(rc={'figure.figsize':(16,9)})
sns.lineplot(x = cons.Hour, y = "Consumption", data = cons);

In [ ]:
sns.set(rc={'figure.figsize':(16,9)})
sns.lineplot(x = cons.Week, y = "Consumption", data = cons);

In [ ]:
sns.set(rc={'figure.figsize':(12,9)})
sns.lineplot(x = cons.Month, y = "Consumption", data = cons);

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(weekly, model='multiplicative', period = 24*7*52) # Since the data is hourly one week is equal to 24*7*52
result.plot()
plt.show();

In [ ]:
plt.title(label='Detrended and Deseasoned Consumption')
plt.xlabel("Year")
plt.ylabel("Consumption/Trend*Seasonality")
plt.legend()
plt.show()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(model_data, order=(3,0,0))
model_fit = model.fit()
model_fit.summary()

In [ ]:
forecast = model_fit.forecast(steps = 24*14)

In [ ]:
errors = test_data - forecast_normalized
err_df = pd.DataFrame(errors, index = test_data.index)
err_df.columns = ["Daily Error"]
err_df.plot()
plt.ylabel("Residuals")
plt.title("Difference of Real Values and Forecasted Values")
plt.legend()
plt.show();

In [ ]:
dmape = err_df["Daily Error"]/ test_data
dmape = pd.DataFrame(dmape)
dmape.columns = ["Daily Mean Absolute Percentage Error"]
dmape

In [ ]:
se_mape = abs(test_data-forecast_normalized)/test_data
ft_actual_sum = test_data.sum()
se_actual_prod_mape = test_data * se_mape
ft_actual_prod_mape_sum = se_actual_prod_mape.sum()
ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum
ft_wmape_forecast

In [ ]:
plt.boxplot(sold);